In [ ]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments
import json
import pandas as pd
from datasets import load_dataset, load_from_disk

In [ ]:
# Load model directly
cache_dir = "/kaggle/working/my_model_dir"
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base", cache_dir=cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-base", cache_dir=cache_dir)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=None)

In [ ]:
ds_train_preprocessed = load_from_disk("/kaggle/input/mathbridge-preprocessed-t5/preprocess_data")

In [ ]:
ds_train_preprocessed

In [ ]:
ds_train_preprocessed.shuffle(seed=42).select(range(100))

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train_preprocessed,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")